# Extract hyperopt data
Extract data from the hyperopt search to be thereafeter analysed in another notebook.

In [1]:
from autotune_advanced_notebook import Benchmarkable
import torch
import io
import matplotlib.pyplot as plt
import seaborn as sb
import matplotlib 
matplotlib.use('TkAgg')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from scvi.dataset.anndataset import AnnDatasetFromAnnData
import scanpy as sc
import numpy as np
import anndata

In [2]:
# Paths to expression data and scVI hyperopt
path_adata='/lustre/groups/ml01/workspace/karin.hrovatin/data/pancreas/scRNA/ref_combined/preprocessed/'
path_data='/lustre/groups/ml01/workspace/karin.hrovatin/data/pancreas/scRNA/ref_combined/scVI/hyperopt/'

### Load data

In [7]:
# Hyperopt folder for which to prepare data
folder_name='1599165129.871496_2000_mll'
# Name used for Benchmarkable
hyperopt_run_name='ref_2000_mll'
# Hvg column name or 'all' to use all genes
hvg_name='highly_variable'

In [8]:
# Load hyperopt result (requires GPU)
hyperopt_res=Benchmarkable(global_path=path_data+folder_name, exp_key='ref', name=hyperopt_run_name)

### Get parameters search results

In [9]:
# Extract all tested params and corresponding metrics - save as tsv into hyperopt folder
params_res=hyperopt_res.get_param_df()
params_res.to_csv(path_data+folder_name+'/trials_ref.tsv',sep='\t',index=False)

### Get latent space of best model

In [19]:
# Load original data to transform to latent space
adata=sc.read_h5ad(path_adata+'data_normalised.h5ad')

In [20]:
# Prepare scVI data object
# Replace X with counts
data=adata.copy()
data.X=data.layers['counts']
del data.layers['counts']
del data.raw
# Select genes
if hvg_name != 'all':
    data=data[:,data.var[hvg_name]]
# Add batch info
batches=data.obs.study_sample.unique()
data.obs['batch_int']=data.obs.study_sample.replace(dict(zip(batches,range(len(batches)))))
# Make scVi data object
data=AnnDatasetFromAnnData(ad = data,batch_label='batch_int')

Trying to set attribute `.obs` of view, copying.


[2020-09-09 14:49:43,251] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2020-09-09 14:49:44,144] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-09-09 14:49:44,148] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-09-09 14:49:48,689] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-09-09 14:49:49,089] INFO - scvi.dataset.dataset | Downsampled from 73487 to 73487 cells


In [21]:
# Get latent space
posterior = hyperopt_res.trainer.create_posterior(hyperopt_res.trainer.model, data, indices=np.arange(len(data)))
latent, batch_indices, labels = posterior.sequential().get_latent()

In [22]:
# Store latent space as adata 
latent_adata = anndata.AnnData(X=latent)
# Add cell metadata
latent_adata.obs=adata.obs.copy()
latent_adata.obs_names=adata.obs_names.copy()

In [23]:
# Compute neighbours and UMAp on latent data
sc.pp.neighbors(latent_adata,n_pcs=0)
sc.tl.umap(latent_adata)

/home/icb/karin.hrovatin/miniconda3/envs/rpy2_3/lib/python3.8/site-packages/scanpy/neighbors/__init__.py:121: FutureWarning: This location for 'distances' is deprecated. It has been moved to .obsp[distances], and will not be accesible here in a future version of anndata.
  adata.uns['neighbors']['distances'] = neighbors.distances
/home/icb/karin.hrovatin/miniconda3/envs/rpy2_3/lib/python3.8/site-packages/scanpy/neighbors/__init__.py:122: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
  adata.uns['neighbors']['connectivities'] = neighbors.connectivities
/home/icb/karin.hrovatin/miniconda3/envs/rpy2_3/lib/python3.8/site-packages/scanpy/tools/_umap.py:147: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
  adata.uns['neighbors']['connectivities'

In [24]:
latent_adata.write(path_data+folder_name+'/latent.h5ad')